In [3]:
# Translate lists of gates to openqasm. Ignores measurement-based uncomputation tricks
# and translates them naively into Toffoli gates.
def to_qasm(gates):
    translate = {
        'cx': 'cx',
        'z': 'z',
        'cz': 'cz',
        'tla_compute': 'ccx',
        'tla_uncompute': 'ccx',
        'ccx': 'ccx',
        's': 's',
        'sdg': 'sdg',
        'h': 'h',
        'x': 'x'
    }
    maxn = 0
    for _, *qs in gates:
        maxn = max(max(qs, default=-1)+1, maxn)
    lines = ["OPENQASM 2.0;", f"qreg q[{maxn}];", "include \"qelib1.inc\";"]
    for name, *qs in gates:
        lines.append(f"{translate[name]} {', '.join(f'q[{q}]' for q in qs)};")
    return '\n'.join(lines)

In [4]:
# QROM-parameterized Jordan-Wigner mapped Majorana operators taken from the SELECT oracle in 
# Lee et al, "Even more efficient quantum computations of chemistry through tensor hypercontraction", arXiv:2011.03494

# Given a phase gradient register of length b, perform a Rz gate of angle pi*n/2^b on
# the target qubit where n is the bitstring store in angle.
def parameterized_rotation(gates: list, angle: list[int], target: int, grad: list[int], ancillas: list[int], tla=False):
    for j in grad:
        gates.append(("cx", target, j))

    carries = []
    cprev, *ancillas = ancillas
    gates.append(("tla_compute", angle[0], grad[0], cprev))
    for i in range(1, len(angle) - 2):
        ci, *ancillas = ancillas
        gates.append(("cx", cprev, grad[i]))
        gates.append(("cx", cprev, angle[i]))
        gates.append(("tla_compute", angle[i], grad[i], ci))
        gates.append(("cx", cprev, ci))
        carries.append(cprev)
        cprev = ci

    if tla:
        return

    gates.append(("cx", ci, grad[-2]))
    gates.append(("cx", ci, angle[-2]))
    gates.append(("z", ci))
    gates.append(("cz", angle[-2], grad[-2]))
    gates.append(("cx", ci, angle[-2]))
    for i in range(len(angle) - 3,0,-1):
        *carries, cprev = carries
        gates.append(("cx", cprev, ci))
        gates.append(("tla_uncompute", angle[i], grad[i], ci))
        gates.append(("cx", cprev, angle[i]))
        ci = cprev
    gates.append(("tla_uncompute", angle[0], grad[0], cprev))

    for i in range(len(angle) - 1):
        gates.append(("cx", angle[i], grad[i]))

    gates.append(("z", angle[-1]))

    for j in grad:
        gates.append(("cx", target, j))

# Perform a givens rotation parameterized by a register on a pair of qubits
def givens_rotation(gates: list, a: int, b: int, angle: list[int], grad: list[int], ancillas: list[int], inverse=False, tla=False):
    gates.extend([
        ("h", a), ("sdg", a), ("h", a),
        ("h", b), ("sdg", b), ("h", b),
        ("cx", b, a),
        ("s", a),
        ("cx", b, a)
    ])

    if not inverse:
        gates.extend([
            ("h", a), ("sdg", a), ("h", a),
            ("h", b), ("s", b), ("h", b)
        ])
    else:
        gates.extend([
            ("h", a), ("s", a), ("h", a),
            ("h", b), ("sdg", b), ("h", b)
        ])

    parameterized_rotation(gates, angle, a, grad, ancillas)
    parameterized_rotation(gates, angle, b, grad, ancillas, tla=tla)

    if tla:
        return

    if not inverse:
        gates.extend([
            ("h", b), ("sdg", b), ("h", b),
            ("h", a), ("s", a), ("h", a),
        ])
    else:
        gates.extend([
            ("h", b), ("s", b), ("h", b),
            ("h", a), ("sdg", a), ("h", a),
        ])

    gates.extend([
        ("cx", b, a),
        ("sdg", a),
        ("cx", b, a),
        ("h", a), ("s", a), ("h", a),
        ("h", b), ("s", b), ("h", b),
    ])

# Change basis of one register of spin-orbitals according to loaded rotations
def basis_change(gates: list, orbitals: list[int], angles: list[list[int]], grad: list[int], ancillas: list[int], inverse=False):
    iterable = list(zip(orbitals, orbitals[1:], angles))
    if not inverse:
        iterable = iterable[::-1]
    for a, b, angle in iterable:
        givens_rotation(gates, a, b, angle, grad, ancillas, inverse=inverse)


# Perform a controlled parameterized majorana operator to a register
def parameterized_majorana(gates: list, control: int, orbitals: list[int], angles: list[list[int]], grad: list[int], ancillas: list[int]):
    basis_change(gates, orbitals, angles, grad, ancillas, inverse=True)
    gates.append(("cz", control, orbitals[0]))
    basis_change(gates, orbitals, angles, grad, ancillas, inverse=False)



In [5]:
# Hamming-weight computation of a binary register, taken from Gidney, "Halving the cost of quantum addition", arXiv:1709.06648

# Compute the Hamming weight of a register, returns a register of size 
# ceil(log(n)) allocated from the ancillas, the remaining ancillas, 
# and a list of gates to uncompute the register
def hamming_weight(gates: list, bits: list[int], ancillas: list[int]):
    fresh = 0
    min_weight = bits.copy()
    next_weight = []
    out = []
    uncompute_gates = []

    while len(min_weight) > 1:
        while len(min_weight) >= 3:
            a, b, c, *min_weight = min_weight
            d = ancillas[fresh]
            fresh += 1
            gates.append(("cx", a, b))
            gates.append(("cx", a, c))
            gates.append(("tla_compute", b, c, d))
            gates.append(("cx", a, b))
            gates.append(("cx", a, d))
            gates.append(("cx", b, c))
            uncompute_gates.append(("cx", a, b))
            uncompute_gates.append(("cx", a, c))
            uncompute_gates.append(("tla_uncompute", b, c, d))
            uncompute_gates.append(("cx", a, b))
            uncompute_gates.append(("cx", a, d))
            uncompute_gates.append(("cx", b, c))
            min_weight.append(c)
            next_weight.append(d)

        if len(min_weight) == 2:
            a, b, *min_weight = min_weight
            c = ancillas[fresh]
            fresh += 1
            gates.append(("tla_compute", a, b, c))
            gates.append(("cx", a, b))
            uncompute_gates.append(("tla_uncompute", a, b, c))
            uncompute_gates.append(("cx", a, b))
            min_weight.append(b)
            next_weight.append(c)

        assert len(min_weight) == 1
        out.append(min_weight[0])
        min_weight = next_weight
        next_weight = []

    out.append(min_weight[0])
    uncompute_gates.reverse()

    return out, ancillas[fresh:], uncompute_gates


In [22]:
# A ripple-carry binary addition circuit, taken from Cuccaro et al, "A new quantum ripple-carry addition circuit", arXiv:quant-ph/0410184

def maj(gates, a, b, c):
    gates.append(("cx", c, b))
    gates.append(("cx", c, a))
    gates.append(("ccx", a, b, c))

def uma(gates, a, b, c, control=None, opt=False):
    if control is None:
        if not opt:
            gates.append(("ccx", a, b, c))
            gates.append(("cx", c, a))
            gates.append(("cx", a, b))
        else:
            gates.append(("ccx", a, b, c))
            gates.append(("cx", c, b))
            gates.append(("cx", a, b))
            gates.append(("cx", c, a))
    else:
        gates.append(("ccx", a, b, c))
        gates.append(("cx", c, b))
        gates.append(("ccx", control, a, b))
        gates.append(("cx", c, a))

def cuccaro_ripple_adder(gates: list, abits: list[int], bbits: list[int], ancillas: list[int], carry_out: int | None = None, control: int | None = None, opt = False):
    if opt:
        *abits, final_a = abits
        *bbits, carry_out = bbits

    c = ancillas[0]
    for (a, b) in zip(abits, bbits):
        maj(gates, c, b, a)
        c = a
    uga = []
    c = ancillas[0]
    for (a, b) in zip(abits, bbits):
        ug = []
        uma(ug, c, b, a, control=control, opt=opt)
        uga.extend(reversed(ug))
        c = a

    if carry_out is not None:
        if control is None:
            gates.append(("cx", abits[-1], carry_out))
        else:
            gates.append(("ccx", control, abits[-1], carry_out))

    if opt:
        gates.append(("cx", final_a, carry_out))

    gates.extend(reversed(uga))


In [12]:
# Output cuccaro_adder_n{} circuits

for n in range(3, 11):
    gates = []
    cuccaro_ripple_adder(gates, list(range(2, 2*n+1, 2)),  list(range(1, 2*n, 2)), [0])
    qasm = to_qasm(gates)
    open(f"output/cuccaro_adder_n{n}.qasm", "w").write(qasm)

In [25]:
# Output hamming_weight_n{} circuits

for n in range(4, 21):
    gates = []
    bits = list(range(n))
    ancillas = list(range(n,2*n))
    _, _, _ = hamming_weight(gates, bits, ancillas)
    qasm = to_qasm(gates)
    open(f"output/hamming_weight_n{n}.qasm", "w").write(qasm)

In [12]:
# Output basis_change_p{}_o{} circuits

for prec in range(4, 11):
    for norbitals in range(2, 6):
        angles = [list(range(prec*i, prec*(i+1))) for i in range(norbitals)]
        orbitals = list(range(prec*norbitals, (prec+1)*norbitals))
        grad = list(range((prec+1)*norbitals, (prec+1)*norbitals + prec))
        ancillas = list(range((prec+1)*norbitals + prec, (prec+1)*norbitals + 2*prec - 2))
        gates = []
        basis_change(gates, orbitals, angles, grad, ancillas, inverse=False)
        qasm = to_qasm(gates)
        open(f"output/basis_change_p{prec}_o{norbitals}.qasm", "w").write(qasm)